### Set up

#### 1. Set  up  accounts and role

In [2]:
!pip install sagemaker==1.39.0

awsebcli 3.12.4 has requirement requests<=2.9.1,>=2.6.1, but you'll have requests 2.20.1 which is incompatible.
spacy 2.1.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.1 which is incompatible.
blis 0.2.4 has requirement numpy>=1.15.0, but you'll have numpy 1.14.1 which is incompatible.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201909140804"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
bucket = "aegovan-data"

In [6]:
trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_less_negative.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_less_negative.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_less_negative.json".format(bucket)
embeddingfile="s3://{}/embeddings/wikipedia-pubmed-and-PMC-w2v.bin.txt".format(bucket)
s3_output_path= "s3://{}/results/".format(bucket)

### Start training

In [7]:
inputs = {
    "train" : trainfile,
    "val" :valfile,
    "embedding" : embeddingfile
}

In [13]:
hyperparameters = {
    "dataset":"PPIDataset",
    "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
    "embeddim":200,
    "batchsize": "32",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "lstmhiddensize": 8,
     "fclayersize": 10,
    "numlayers":3,
    "poolingkernelsize":3
   
}

In [14]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [10]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
              'commit': 'c050a2818efd6dc6d2adb3dac07c102b94c92eab'}

In [15]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    base_job_name ="ppi-extractor")

In [16]:
estimator.fit(inputs)

2019-09-19 08:29:02 Starting - Starting the training job...
2019-09-19 08:29:03 Starting - Launching requested ML instances...
2019-09-19 08:30:00 Starting - Preparing the instances for training......
2019-09-19 08:30:57 Downloading - Downloading input data.....................
2019-09-19 08:35:37 Training - Downloading the training image..........bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-09-19 08:37:41,948 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-09-19 08:37:41,992 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-09-19 08:37:45,006 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-09-19 08:37:45,263 sagemaker-containers INFO     Module main_train does not provide a setup.py. 
Generating setup.py
2019-09-19 08:37:45,264 sagemaker-containers INFO     Generating setup.cfg
2019-09-19 08:37:4


2019-09-19 08:37:40 Training - Training image download completed. Training in progress.2019-09-19 08:45:11,084 - algorithms.PretrainedEmbedderLoader - INFO - The number of words intialised without embbeder is 127
2019-09-19 08:46:32,238 - algorithms.transform_protein_mask - INFO - Running TransformProteinMask 
2019-09-19 08:46:36,535 - algorithms.transform_protein_mask - INFO - Completed TransformProteinMask 
2019-09-19 08:46:36,535 - algorithms.transform_protein_mask - INFO - Running TransformProteinMask 
2019-09-19 08:46:36,544 - algorithms.transform_protein_mask - INFO - Completed TransformProteinMask 
2019-09-19 08:46:36,544 - algorithms.transform_sentence_tokeniser - INFO - Running sentence tokenisor 
2019-09-19 08:48:35,094 - algorithms.transform_sentence_tokeniser - INFO - Completed  sentence tokenisor 
2019-09-19 08:48:35,100 - algorithms.transform_text_index - INFO - Transforming TransformTextToIndex
2019-09-19 08:48:35,617 - algorithms.transform_text_index - INFO - Completed

KeyboardInterrupt: 

### HPO

In [17]:
objective_metric_name ="ValidationFScore"

In [18]:
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, CategoricalParameter, ContinuousParameter
hyperparameter_ranges = {'lstmhiddensize': IntegerParameter(2,200), #ContinuousParameter(0.01, 0.2),
                        "fclayersize": IntegerParameter(2,50),
                        "numlayers":IntegerParameter(1,10),
                        "poolingkernelsize":IntegerParameter(2,10)
                        }

In [25]:
hyperparameters={ "trainfile":trainfile.split("/")[-1],
    "valfile":valfile.split("/")[-1],
    "embeddingfile":embeddingfile.split("/")[-1],
                        "embeddim":"200",
                        "dataset":"PPIDataset",
                 "earlystoppingpatience":20,
                         'epochs': 100}

In [26]:

estimator = PyTorch(
     entry_point='main_train.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                   # git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
    
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    base_job_name ="hpo-ppi-extractor")

In [28]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=50,
                            max_parallel_jobs=4,
                            strategy="Random",
                            base_tuning_job_name="hpo-ppi-extractor")
tuner.fit(inputs)